<a href="https://colab.research.google.com/github/dtabuena/CrispyMice/blob/main/Run_All_Crispy_Recs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)



!pip install openpyxl
!pip install XlsxWriter


'''Get Repositories'''
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
try: shutil.rmtree('/content/CrispyMice')
except: None
# !git clone https://github.com/dtabuena/CrispyMice
# to_import = ['CrispyPipelineFuncs.ipynb']
# for i in to_import:
#     f = '/content/CrispyMice/' + i
#     %run $f
!git clone https://github.com/dtabuena/EphysLib
to_import = [
            'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            'Simple_ABF_tools.ipynb',
            'fun_math.ipynb',
            'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'Analyzers/input_resistance_analyzer.ipynb',
            'Analyzers/gain_analyzer.ipynb',
            'Analyzers/latencey_analyzer.ipynb',
            'Analyzers/IV_analyzer.ipynb',
            'Analyzers/Vm_analyzer.ipynb',
            'Analyzers/membrane_analyzer.ipynb',
            'Analyzers/rheobase_analyzer.ipynb',
            'Ephys_wrapper.ipynb',
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f




In [ ]:
##### Setup Protocol List
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree'
           'I0 - 3min GapFree']

In [ ]:
# dataset = {'data_name': 'RNF182',
#            'data_source': "https://www.dropbox.com/sh/n9t8p257wnzlijk/AAC9Z36JodisyZjnM3mkJC3Xa?dl=0",
#            'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
#            }

# RNF182_results = crispy_mouse_wrap(dataset,VC_prot,IC_prot,strat_cols=['Rec_date', 'Cell_Type'])

In [ ]:
# dataset = {'data_name': 'MGDA2',
#            'data_source': "https://www.dropbox.com/sh/bj5ke9c1pqwpsct/AACqznMXzJj7epVBVDaSifnma?dl=0",
#            'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
#            }

# MGDA2_results = crispy_mouse_wrap(dataset,VC_prot,IC_prot,strat_cols=['Rec_date', 'Cell_Type'])

In [ ]:
# dataset = {'data_name': 'NELL2_DG',
#            'data_source': "https://www.dropbox.com/sh/46zwrax2s9kh8ms/AAACswSGHMFBpYp8t_P4JrHHa?dl=0",
#            'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
#            }
# # 
# NELL2_results = crispy_mouse_wrap(dataset,VC_prot,IC_prot,strat_cols=['Rec_date', 'Cell_Type'])

In [ ]:
dataset = {'data_name': 'NELL2_CA3',
           'data_source': "https://www.dropbox.com/sh/kdyo8a4gy4krpp8/AAD-ra8gZJpdsEKq-jk-Yk_da?dl=0",
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }

NELL2_results = ephys_wrapper(dataset,VC_prot,IC_prot,strat_cols=['Rec_date', 'Cell_Type'])

In [ ]:
single_val_strat_groups = ['ap_amplitutude',
                            'Rmp_mV',
                            'Ra_160.0',
                            'Rm_160.0',
                            'Cm_pc_10.0',                    
                            'Cmq_160.0',
                            'Ra_160.0',
                            'Rm_160.0',
                            'Gain_(HzpA)',
                            'Gain_(%pA)',
                            'Rheobase',
                            'AP_thresh_US',
                            'fast_after_hyperpol',
                            'Spike_latency_(ms)',
                            'Input_Resistance_MO']


strat_df_dict = NELL2_results['strat_df_dict'].copy()
file_naming_scheme = ['Rec_date', 'Virus', 'GenoType', 'Sex', 'Age', 'Slice_Num', 'Cell_num', 'Cell_Type']
filtered_dict, fail_dict = final_qc(strat_df_dict,file_naming_scheme,qc_Rmp=-45,qc_RR=.35,qc_Ra=60)
alt_strat_dict = restratify_results(filtered_dict,file_naming_scheme,single_val_strat_groups)
resp_curve_list = ['IV_Early_(V_stim)','IV_Early_(I_peak)','IV_Steady_State_(I_mean)','Stim_Levels_(pA)','Spike_Counts']
response_curve_data = stratify_response_curve(filtered_dict,resp_curve_list,strat_list=['POS','NEG'])
alt_strat_dict.update(response_curve_data)

write_strat_dfs(alt_strat_dict, dataset['data_name']+'_results_stratified_alternate')
